
Ignoring Warnings




In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imoorting necessary libraries

In [0]:


import yaml
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from PIL import Image 
from google.colab.patches import cv2_imshow
import csv

from sklearn.metrics import roc_auc_score


mounting with google drive



In [4]:
# mounting with google drive

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Path from Google Drive

In [0]:
data_path='/content/drive/My Drive/Colab Notebooks/CXR/'

Uploading Files

In [0]:
ft = pd.read_csv(data_path+'CheXpert-v1.0-small/train.csv')
fv = pd.read_csv(data_path+'CheXpert-v1.0-small/valid.csv')

Analysing dimensions of data 

In [7]:
ft.size

4244866

In [8]:
ft.shape

(223414, 19)

Only considering first  2000 images for Learning

In [9]:
ft.head(5)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


Anlysing the available Parameters and Classes. 

In [0]:
chexnet_targets = ['No Finding','Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity','Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture','Support Devices']
chexpert_targets = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

In [11]:
print(chexnet_targets)
print(chexpert_targets)

['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']
['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']


In paper we are having various scenarios for uncertain values depicted as u. We are taking them to be one for 'Atelectasis','Edema' and zero for 'Cardiomegaly', 'Consolidation', 'Pleural Effusion'. This values are taken after analysisng the paper. This value gives best result of all combinations as suggested.

In [0]:
u_one_features = ['Atelectasis', 'Edema']
u_zero_features = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']

This function is to determine what problem is patient affected from 

In [0]:
def feature_string(row):
    feature_list = []
    for feature in u_one_features:
        if row[feature] in [-1,1]:
            feature_list.append(feature)
            
    for feature in u_zero_features:
        if row[feature] == 1:
            feature_list.append(feature)
            
    return feature_list

Addding feature weather the data is validity set or  treaining set.

In [17]:
ft.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [19]:
fv.head(2)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Adding patient number and study feature for both training and validity data



In [0]:
ft['patient'] = ft.Path.str.split('/',3,True)[2]
ft['study'] = ft.Path.str.split('/',4,True)[3]

fv['patient'] = fv.Path.str.split('/',3,True)[2]
fv ['study'] = fv.Path.str.split('/',4,True)[3]

In [22]:
ft.head(2)
ft['feature_string'] = ft.apply(feature_string,axis = 1).fillna('')
ft.head()
ft['Path'][0]


'CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg'

In [24]:
ft.head(10)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,study,feature_string
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,patient00001,study1,[]
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,patient00002,study2,"[Atelectasis, Edema]"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002,study1,[]
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002,study1,[]
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient00003,study1,[Edema]
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,patient00004,study1,[]
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,patient00004,study1,[]
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,patient00005,study1,[]
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,patient00005,study1,[]
9,CheXpert-v1.0-small/train/patient00005/study2/...,Male,33,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,patient00005,study2,[]


In [25]:

fv['feature_string'] = fv.apply(feature_string,axis = 1).fillna('')
fv.head()
fv['Path'][0]

'CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg'

In [29]:
print(len(ft['feature_string'][1]))
ft['feature_string'][1]

2


['Atelectasis', 'Edema']

In [0]:
print(len(fv['feature_string'][1]))
fv['feature_string'][1]

In [30]:
ft.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,study,feature_string
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,patient00001,study1,[]
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,patient00002,study2,"[Atelectasis, Edema]"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002,study1,[]
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002,study1,[]
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient00003,study1,[Edema]


In [34]:
li=[]
li2=[]
for i in range(len(ft)):
  if len(ft['feature_string'][i])==0:
    li.append(ft['Path'][i])
    li2.append('n')

  for j in range(len(ft['feature_string'][i])):
    li.append(ft['Path'][i])
    li2.append(ft['feature_string'][i][j])
print(len(li))
print(len(li2))


li3=[]
li4=[]
for i in range(len(fv)):
  if len(fv['feature_string'][i])==0:
    li3.append(fv['Path'][i])
    li4.append('n')

  for j in range(len(fv['feature_string'][i])):
    li3.append(fv['Path'][i])
    li4.append(fv['feature_string'][i][j])
print(len(li3))
print(len(li4))

326448
326448
390
390


In [0]:
import numpy as np 
import cv2 
from google.colab.patches import cv2_imshow
from keras.preprocessing.image import load_img  

# You can give path to the 
# image as first argument 
path1='/content/drive/My Drive/Colab Notebooks/'


Xtrain=[]
for i in range(len(li)):
  img = load_img(path1+li[i], target_size=(224, 224))
  Xtrain.append(img)


Xtest=[]
for i in range(len(li3)):
  img = load_img(path1+li3[i], target_size=(224, 224))
  Xtest.append(img)










234


In [0]:

X_train=np.zeros((len(li),224,224,3))

for i in range(len(li)):
  X_train[i]=np.array(Xtrain[i])



X_test=np.zeros((len(li3),224,224,3))

for i in range(len(li3)):
  X_test[i]=np.array(Xtest[i])








In [0]:
# u_one_features = ['Atelectasis', 'Edema']
# u_zero_features = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']

In [0]:
Y_train=np.zeros((len(li2)))
for i in range(len(li2)):
  if li2[i]=='Atelectasis':
    Y_train[i]=1;
  elif li2[i]=='Edema':
    Y_train[i]=2;  
  elif li2[i]=='Cardiomegaly':
    Y_train[i]=3;
  elif li2[i]=='Consolidation':
    Y_train[i]=4;
  elif li2[i]=='Pleural Effusion':
    Y_train[i]=5;
  elif li2[i]=='n':
    Y_train[i]=6;      



Y_test=np.zeros((len(li4)))
for i in range(len(li4)):
  if li4[i]=='Atelectasis':
    Y_test[i]=1;
  elif li4[i]=='Edema':
    Y_test[i]=2;  
  elif li4[i]=='Cardiomegaly':
    Y_test[i]=3;
  elif li4[i]=='Consolidation':
    Y_test[i]=4;
  elif li4[i]=='Pleural Effusion':
    Y_test[i]=5;
  elif li4[i]=='n':
    Y_test[i]=6;      



In [0]:
X_t=X_train
Y_t=Y_train
X_te=X_test
Y_te=Y_test




In [0]:
import keras
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D
from keras.utils import plot_model



In [0]:
# tf.keras.applications.DenseNet121(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
# )
from keras.applications import DenseNet121
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))

model = DenseNet121(input_tensor=input_tensor, weights='imagenet', include_top=True)

In [0]:

model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.fit(X_t,Y_t,batch_size=20,epochs=50,validation_split=0.1,shuffle=True)



Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [0]:
model.evaluate(X_te, Y_te)

89/89 [==============================] - 1s 13ms/step


[2.166935462630197, 0.3258427083492279]

In [0]:
from keras.models import Model
import scipy as sp
gap_weights = model.layers[-1].get_weights()[0]
gap_weights.shape

(1024, 1000)

In [0]:
cam_model  = Model(inputs=model.input,outputs=(model.layers[-3].output,model.layers[-1].output))

In [0]:
features,results = cam_model.predict(X_te)
features.shape

(89, 230, 230, 3)

In [0]:
for idx in range(2):
    features_for_one_img = features[idx,:,:,:]
    #height_roomout = X_train.shape[1]/features_for_one_img.shape[0]
    #width_roomout  = X_train.shape[2]/features_for_one_img.shape[1]
    #print(height_roomout,width_roomout)
 
    #cam_features = sp.ndimage.zoom(features_for_one_img, (height_roomout, width_roomout, 1), order=2)
    #print(cam_features.shape)
    pred = np.argmax(results[idx])
    print(pred)
    cam_features = features_for_one_img
    
    
    plt.figure(facecolor='white')
    cam_weights = gap_weights[:,pred]
    cam_output  = np.dot(cam_features,cam_weights)
    print(features_for_one_img.shape)

    buf = 'Predicted Class = ' +str( pred )+ ', Probability = ' + str(results[idx][pred])

    plt.xlabel(buf)

    plt.imshow(X_te[idx],alpha=0.5)

    plt.imshow(cam_output, cmap='jet',alpha=0.5)

     

    plt.show()